In [2]:
import pandas as pd
import faiss as fa
from sentence_transformers import SentenceTransformer 
import numpy as np

In [37]:
from langchain_openai import ChatOpenAI
from openai import OpenAI

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [5]:
chat = ChatOpenAI(api_key= os.environ.get("OPENAI_API_KEY"), model = "gpt-4o")

PAYPAL EMAIL EXAMPLES (PURCHASE + REFUND)

In [6]:
paypal_email_body1 = """
You sent a payment of $42.15 USD to Spotify USA Inc.

Transaction ID: 7TR837392H8491234
Transaction date: July 15, 2025
Payment method: Visa ending in 1234
"""

paypal_email_meta1 = {
    "message_id": "m1",
    "subject": "Receipt for Your Payment to Spotify",
    "received_ts": "2025-07-15 10:22:00-0700"
}
paypal_email_body2 = """
We’ve issued a full refund of $42.15 USD to your PayPal balance for the following transaction:
Merchant: Spotify USA Inc.

Original Transaction ID: 7TR837392H8491234
Refund ID: 1RB837392H8495678
Refund date: July 22, 2025
"""

paypal_email_meta2 = {
    "message_id": "paypal_refund1",   # unique Gmail message ID
    "subject": "Your refund has been issued",  # from the email header
    "received_ts": "2025-07-22 09:15:00-0700"  # when Gmail received the email
}

In [7]:
import re
import pandas as pd

def normalize_to_utc(date_str):
    """
    Try to parse a date like 'July 15, 2025' into UTC ISO format.
    If parsing fails, return None.
    """
    try:
        ts = pd.to_datetime(date_str, utc=True, errors="coerce")
        if pd.isna(ts):
            return None
        return ts.strftime("%Y-%m-%dT%H:%M:%SZ")
    except Exception:
        return None


result = normalize_to_utc('September 15, 1998')
print(result)


1998-09-15T00:00:00Z


In [8]:
def parse_paypal(email_body: str, email_meta: dict) -> dict:
    """
    Parse a PayPal purchase or refund email into your transaction schema.
    email_meta should include keys: message_id, subject, received_ts
    """
    txn = {
        "id": None,
        "provider": "paypal",
        "type": None,
        "amount": None,
        "currency": None,
        "counterparty": None,
        "category": None,
        "status": None,
        "txn_time_utc": None,
        "order_id": None,
        "last4": None,
        "memo": "",
        "email_message_id": email_meta.get("message_id"),
        "subject": email_meta.get("subject"),
        "received_ts": normalize_to_utc(email_meta.get("received_ts")),
        "raw_text": email_body.strip()[:5000]  # trim long bodies
    }

    # --- Type ---
    if "you sent a payment" in email_body.lower():
        txn["type"] = "purchase"
    elif "refund" in email_body.lower():
        txn["type"] = "refund"
    else:
        txn["type"] = "unknown"

    # --- Amount & Currency ---
    m_amt = re.search(r"\$([0-9]+\.[0-9]{2})\s*([A-Z]{3})?", email_body)
    if m_amt:
        txn["amount"] = float(m_amt.group(1))
        txn["currency"] = m_amt.group(2) if m_amt.group(2) else "USD"

    # --- Counterparty ---
    if txn["type"] == "purchase":
        m_to = re.search(r"to ([A-Za-z0-9 .,&-]+)", email_body)
    elif txn["type"] == "refund":
        m_to = re.search(r"Merchant:\s*(.+)", email_body)
    if m_to:
        txn["counterparty"] = m_to.group(1).strip()
    else:
        txn["counterparty"] = "unknown"

    # --- Transaction date ---
    if txn["type"] == "purchase":
        m_date = re.search(r"Transaction date:\s*(.*)", email_body)
    elif txn["type"] == "refund":
        m_date = re.search(r"Refund date:\s*(.*)", email_body)
    if m_date:
        txn["txn_time_utc"] = normalize_to_utc(m_date.group(1))


    # --- Order ID ---  
    if txn["type"] == "purchase":
        m_tid = re.search(r"Transaction ID:\s*([A-Z0-9]+)", email_body)
    elif txn["type"] == "refund":
        m_tid = re.search(r"Refund ID:\s*(.+)", email_body)
    if m_tid:
        txn["order_id"] = m_tid.group(1)

    # --- Last 4 digits ---
    m_last4 = re.search(r"ending in (\d{4})", email_body)
    if m_last4:
        txn["last4"] = m_last4.group(1)

    # --- Status ---
    txn["status"] = "completed"

    # --- Unique ID ---
    if txn["order_id"]:
        txn["id"] = "paypal_" + txn["order_id"]
    else:
        txn["id"] = "paypal_" + str(txn["email_message_id"])

    return txn

paypalresultemail1 = parse_paypal(paypal_email_body1, paypal_email_meta1)
paypalresultemail2 = parse_paypal(paypal_email_body2, paypal_email_meta2)
print(paypalresultemail1)
print(paypalresultemail2)

{'id': 'paypal_7TR837392H8491234', 'provider': 'paypal', 'type': 'purchase', 'amount': 42.15, 'currency': 'USD', 'counterparty': 'Spotify USA Inc.', 'category': None, 'status': 'completed', 'txn_time_utc': '2025-07-15T00:00:00Z', 'order_id': '7TR837392H8491234', 'last4': '1234', 'memo': '', 'email_message_id': 'm1', 'subject': 'Receipt for Your Payment to Spotify', 'received_ts': '2025-07-15T17:22:00Z', 'raw_text': 'You sent a payment of $42.15 USD to Spotify USA Inc.\n\nTransaction ID: 7TR837392H8491234\nTransaction date: July 15, 2025\nPayment method: Visa ending in 1234'}
{'id': 'paypal_1RB837392H8495678', 'provider': 'paypal', 'type': 'refund', 'amount': 42.15, 'currency': 'USD', 'counterparty': 'Spotify USA Inc.', 'category': None, 'status': 'completed', 'txn_time_utc': '2025-07-22T00:00:00Z', 'order_id': '1RB837392H8495678', 'last4': None, 'memo': '', 'email_message_id': 'paypal_refund1', 'subject': 'Your refund has been issued', 'received_ts': '2025-07-22T16:15:00Z', 'raw_text':

VENMO EMAIL EXAMPLES (TRANSFER IN + OUT)

In [9]:
venmo_email_body1= """ 
You paid John Doe $25.00 on July 20, 2025
Note: Sushi dinner 🍣
"""

venmo_email_meta1 = {
    "message_id": "venmo1",
    "subject": "You paid John Doe",
    "received_ts": "2025-07-20 12:00:00"   # fake received time
}

venmo_email_body2 = """
Jane Smith paid you $50.00 on July 19, 2025
Note: Rent for July
"""

venmo_email_meta2 = {
    "message_id": "venmo2",
    "subject": "Jane Smith paid you $50.00",
    "received_ts": "2025-07-19 14:30:00"
}

In [10]:
def parse_venmo(email_body, email_meta):
    txn = {
       "id": None,
        "provider": "venmo",
        "type": None,
        "amount": None,
        "currency": None,
        "counterparty": None,
        "category": None,
        "status": None,
        "txn_time_utc": None,
        "order_id": None,
        "last4": None,
        "memo": "",
        "email_message_id": email_meta.get("message_id"),
        "subject": email_meta.get("subject"),
        "received_ts": normalize_to_utc(email_meta.get("received_ts")),
        "raw_text": email_body.strip()[:5000]  # trim long bodies
    }
     # --- Type ---
    if "you paid" in email_body.lower():
        txn["type"] = "transfer_out"
    elif "paid you" in email_body.lower():
        txn["type"] = "transfer_in"
    else:
        txn["type"] = "unknown"

    # --- Amount & Currency ---
    m_amt = re.search(r"\$([0-9]+\.[0-9]{2})", email_body)
    if m_amt:
        txn["amount"] = float(m_amt.group(1))
        txn["currency"] = "USD"

       # --- Counterparty ---
    if txn["type"] == "transfer_out":
        m_to = re.search(r"You paid (.+?) \$", email_body)
        if m_to:
            txn["counterparty"] = m_to.group(1).strip()
    elif txn["type"] == "transfer_in":
        m_from = re.search(r"(.+?) paid you", email_body)
        if m_from:
            txn["counterparty"] = m_from.group(1).strip()
    else:
        txn["counterparty"] = "unknown"
    
    # --- Transaction date ---
    m_date = re.search(r"on (.+? \d{1,2}, \d{4})", email_body)
    if m_date:
        txn["txn_time_utc"] = normalize_to_utc(m_date.group(1))
         
    
    #Memo
    m_memo = re.search(r"Note:\s*(.+)", email_body)
    if m_memo:
        txn["memo"] = m_memo.group(1).strip()

    # Step 7: Fill defaults
    txn["status"] = "completed"
    txn["category"] = None
    txn["order_id"] = None
    txn["last4"] = None

    # Step 8: Generate unique id
    txn["id"] = "venmo_" + txn["email_message_id"]

    return txn
venmoresultemail1  = parse_venmo(venmo_email_body1,venmo_email_meta1)
venmoresultemail2 = parse_venmo(venmo_email_body2, venmo_email_meta2)
print(venmoresultemail1)
print(venmoresultemail2)


{'id': 'venmo_venmo1', 'provider': 'venmo', 'type': 'transfer_out', 'amount': 25.0, 'currency': 'USD', 'counterparty': 'John Doe', 'category': None, 'status': 'completed', 'txn_time_utc': '2025-07-20T00:00:00Z', 'order_id': None, 'last4': None, 'memo': 'Sushi dinner 🍣', 'email_message_id': 'venmo1', 'subject': 'You paid John Doe', 'received_ts': '2025-07-20T12:00:00Z', 'raw_text': 'You paid John Doe $25.00 on July 20, 2025\nNote: Sushi dinner 🍣'}
{'id': 'venmo_venmo2', 'provider': 'venmo', 'type': 'transfer_in', 'amount': 50.0, 'currency': 'USD', 'counterparty': 'Jane Smith', 'category': None, 'status': 'completed', 'txn_time_utc': '2025-07-19T00:00:00Z', 'order_id': None, 'last4': None, 'memo': 'Rent for July', 'email_message_id': 'venmo2', 'subject': 'Jane Smith paid you $50.00', 'received_ts': '2025-07-19T14:30:00Z', 'raw_text': 'Jane Smith paid you $50.00 on July 19, 2025\nNote: Rent for July'}


STORE IN DATAFRAME FOR EASIER FUTURE FILTERING

In [11]:
#store in dataframe maybe put in database later
df = pd.DataFrame([paypalresultemail1, paypalresultemail2, venmoresultemail1, venmoresultemail2])
display(df)

,id,provider,type,amount,currency,counterparty,category,status,txn_time_utc,order_id,last4,memo,email_message_id,subject,received_ts,raw_text
0,paypal_7TR837392H8491234,paypal,purchase,42.15,USD,Spotify USA Inc.,None,completed,2025-07-15T00:00:00Z,7TR837392H8491234,1234,,m1,Receipt for Your Payment to Spotify,2025-07-15T17:22:00Z,You sent a payment of $42.15 USD to Spotify US...
1,paypal_1RB837392H8495678,paypal,refund,42.15,USD,Spotify USA Inc.,None,completed,2025-07-22T00:00:00Z,1RB837392H8495678,None,,paypal_refund1,Your refund has been issued,2025-07-22T16:15:00Z,We’ve issued a full refund of $42.15 USD to yo...
2,venmo_venmo1,venmo,transfer_out,25.00,USD,John Doe,None,completed,2025-07-20T00:00:00Z,None,None,Sushi dinner 🍣,venmo1,You paid John Doe,2025-07-20T12:00:00Z,"You paid John Doe $25.00 on July 20, 2025\nNot..."
3,venmo_venmo2,venmo,transfer_in,50.00,USD,Jane Smith,None,completed,2025-07-19T00:00:00Z,None,None,Rent for July,venmo2,Jane Smith paid you $50.00,2025-07-19T14:30:00Z,"Jane Smith paid you $50.00 on July 19, 2025\nN..."


FILTERING AND AGGREGATION TESTING (LIKELY FOR METADATA FILTERING LATER ON)

In [12]:
venmo = df[df["provider"] == "venmo"]
display(venmo)

,id,provider,type,amount,currency,counterparty,category,status,txn_time_utc,order_id,last4,memo,email_message_id,subject,received_ts,raw_text
2,venmo_venmo1,venmo,transfer_out,25.0,USD,John Doe,None,completed,2025-07-20T00:00:00Z,None,None,Sushi dinner 🍣,venmo1,You paid John Doe,2025-07-20T12:00:00Z,"You paid John Doe $25.00 on July 20, 2025\nNot..."
3,venmo_venmo2,venmo,transfer_in,50.0,USD,Jane Smith,None,completed,2025-07-19T00:00:00Z,None,None,Rent for July,venmo2,Jane Smith paid you $50.00,2025-07-19T14:30:00Z,"Jane Smith paid you $50.00 on July 19, 2025\nN..."


In [13]:
refundpaypal = df[(df["provider"] == "paypal") & (df["type"] == "refund")]
display(refundpaypal)

,id,provider,type,amount,currency,counterparty,category,status,txn_time_utc,order_id,last4,memo,email_message_id,subject,received_ts,raw_text
1,paypal_1RB837392H8495678,paypal,refund,42.15,USD,Spotify USA Inc.,None,completed,2025-07-22T00:00:00Z,1RB837392H8495678,None,,paypal_refund1,Your refund has been issued,2025-07-22T16:15:00Z,We’ve issued a full refund of $42.15 USD to yo...


In [14]:
sumvenmo = df[df["provider"] == "venmo"]["amount"].sum()
display(sumvenmo)

np.float64(75.0)

In [15]:
maxamount = df.loc[df["amount"].idxmax()]
display(maxamount)

id                                                       venmo_venmo2
provider                                                        venmo
type                                                      transfer_in
amount                                                           50.0
currency                                                          USD
counterparty                                               Jane Smith
category                                                         None
status                                                      completed
txn_time_utc                                     2025-07-19T00:00:00Z
order_id                                                         None
last4                                                            None
memo                                                    Rent for July
email_message_id                                               venmo2
subject                                    Jane Smith paid you $50.00
received_ts         

In [16]:
spotify = df[df["counterparty"].str.contains("Spotify")]
display(spotify)


,id,provider,type,amount,currency,counterparty,category,status,txn_time_utc,order_id,last4,memo,email_message_id,subject,received_ts,raw_text
0,paypal_7TR837392H8491234,paypal,purchase,42.15,USD,Spotify USA Inc.,None,completed,2025-07-15T00:00:00Z,7TR837392H8491234,1234,,m1,Receipt for Your Payment to Spotify,2025-07-15T17:22:00Z,You sent a payment of $42.15 USD to Spotify US...
1,paypal_1RB837392H8495678,paypal,refund,42.15,USD,Spotify USA Inc.,None,completed,2025-07-22T00:00:00Z,1RB837392H8495678,None,,paypal_refund1,Your refund has been issued,2025-07-22T16:15:00Z,We’ve issued a full refund of $42.15 USD to yo...


In [17]:
net_spotify = spotify.apply(
    lambda row: row["amount"] if row["type"] == "purchase" else -row["amount"], axis=1
).sum()
display(net_spotify)

np.float64(0.0)

In [18]:
by_provider = df.groupby("provider")["amount"].sum()
display(by_provider)

provider
paypal    84.3
venmo     75.0
Name: amount, dtype: float64

In [19]:
df["direction"] = df["type"].map({
    "purchase": "out",
    "refund": "in",
    "transfer_out": "out",
    "transfer_in": "in"
})

summary = df.groupby("direction")["amount"].sum()
display(summary)

direction
in     92.15
out    67.15
Name: amount, dtype: float64

In [20]:
biggest = df.groupby("counterparty")["amount"].sum().sort_values(ascending=False)
display(biggest)

counterparty
Spotify USA Inc.    84.3
Jane Smith          50.0
John Doe            25.0
Name: amount, dtype: float64

In [21]:
display(df)

,id,provider,type,amount,currency,counterparty,category,status,txn_time_utc,order_id,last4,memo,email_message_id,subject,received_ts,raw_text,direction
0,paypal_7TR837392H8491234,paypal,purchase,42.15,USD,Spotify USA Inc.,None,completed,2025-07-15T00:00:00Z,7TR837392H8491234,1234,,m1,Receipt for Your Payment to Spotify,2025-07-15T17:22:00Z,You sent a payment of $42.15 USD to Spotify US...,out
1,paypal_1RB837392H8495678,paypal,refund,42.15,USD,Spotify USA Inc.,None,completed,2025-07-22T00:00:00Z,1RB837392H8495678,None,,paypal_refund1,Your refund has been issued,2025-07-22T16:15:00Z,We’ve issued a full refund of $42.15 USD to yo...,in
2,venmo_venmo1,venmo,transfer_out,25.00,USD,John Doe,None,completed,2025-07-20T00:00:00Z,None,None,Sushi dinner 🍣,venmo1,You paid John Doe,2025-07-20T12:00:00Z,"You paid John Doe $25.00 on July 20, 2025\nNot...",out
3,venmo_venmo2,venmo,transfer_in,50.00,USD,Jane Smith,None,completed,2025-07-19T00:00:00Z,None,None,Rent for July,venmo2,Jane Smith paid you $50.00,2025-07-19T14:30:00Z,"Jane Smith paid you $50.00 on July 19, 2025\nN...",in


CONVERTING ROWS IN DF TO STRING TO EMBED INTO FAISS

In [30]:
def row_to_str (row):
    rowstr = ""
    if row['provider']=='paypal':
        if row['type'] == 'purchase':
            rowstr = f"{row['provider']} {row['type']} of {row['amount']:.2f} {row['currency']} to {row['counterparty']} on {row['txn_time_utc']} (Visa Ending in {row['last4']})"
        elif row['type'] == 'refund':
            rowstr = f"{row['provider']} {row['type']} of {row['amount']:.2f} {row['currency']} from {row['counterparty']} on {row['txn_time_utc']}"
    elif row['provider']=='venmo':
        note = ""
        if row['memo']:
            note = f"(note: {row['memo']})"
        else:
            note = ""
        if row['type'] == 'transfer_out':
            rowstr = f"{row['provider']} payment of {row['amount']:.2f} {row['currency']} to {row['counterparty']} on {row['txn_time_utc']} {note}" 
        elif row['type'] == 'transfer_in':
            rowstr = f"{row['provider']} payment of {row['amount']:.2f} {row['currency']} received from {row['counterparty']} on {row['txn_time_utc']} {note}" 
    return rowstr.strip() #removes leading or trailing spaces

allrowstr = [row_to_str(row) for idx, row in df.iterrows()]
print(allrowstr)


['paypal purchase of 42.15 USD to Spotify USA Inc. on 2025-07-15T00:00:00Z (Visa Ending in 1234)', 'paypal refund of 42.15 USD from Spotify USA Inc. on 2025-07-22T00:00:00Z', 'venmo payment of 25.00 USD to John Doe on 2025-07-20T00:00:00Z (note: Sushi dinner 🍣)', 'venmo payment of 50.00 USD received from Jane Smith on 2025-07-19T00:00:00Z (note: Rent for July)']


EMBEDDING ROW STRINGS INTO FAISS

In [23]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [24]:
embeddings = model.encode(allrowstr)
embeddings = np.array(embeddings).astype("float32")
dim = embeddings.shape[1]   # should be 384
index = fa.IndexFlatL2(dim)
index.add(embeddings)



RETRIEVAL FUNCTION (WRAP BELOW INTO FUNCTION)

In [28]:
def query_transactions(query, top_k=2):
    # Embed the query
    query_emb = model.encode([query]).astype("float32")

    # Search FAISS
    D, I = index.search(query_emb, top_k)

    results = []
    for rank, idx in enumerate(I[0]):
        results.append({
            "index": idx,                        # DataFrame index
            "summary": allrowstr[idx],           # summary string
            "row": df.iloc[idx].to_dict(),       # full structured row
            "score": float(D[0][rank])           # similarity distance
        })
    return results
results = query_transactions("refund from spotify")
print(results)

[{'index': np.int64(1), 'summary': 'paypal refund of 42.15 USD from Spotify USA Inc. on 2025-07-22T00:00:00Z', 'row': {'id': 'paypal_1RB837392H8495678', 'provider': 'paypal', 'type': 'refund', 'amount': 42.15, 'currency': 'USD', 'counterparty': 'Spotify USA Inc.', 'category': None, 'status': 'completed', 'txn_time_utc': '2025-07-22T00:00:00Z', 'order_id': '1RB837392H8495678', 'last4': None, 'memo': '', 'email_message_id': 'paypal_refund1', 'subject': 'Your refund has been issued', 'received_ts': '2025-07-22T16:15:00Z', 'raw_text': 'We’ve issued a full refund of $42.15 USD to your PayPal balance for the following transaction:\nMerchant: Spotify USA Inc.\n\nOriginal Transaction ID: 7TR837392H8491234\nRefund ID: 1RB837392H8495678\nRefund date: July 22, 2025', 'direction': 'in'}, 'score': 0.43438786268234253}, {'index': np.int64(0), 'summary': 'paypal purchase of 42.15 USD to Spotify USA Inc. on 2025-07-15T00:00:00Z (Visa Ending in 1234)', 'row': {'id': 'paypal_7TR837392H8491234', 'provide

In [ ]:
def generate_prompt(query, top_k=2):
    # Step 1: Retrieve candidate transactions
    retrieved = query_transactions(query, top_k=top_k)

    # Step 2: Build context string (summaries only)
    context = "\n".join([r["summary"] for r in retrieved])

    # Step 3: Construct prompt
    prompt = f"""
User question: {query}

Transactions:
{context}
"""
    return prompt.strip(), retrieved

print(generate_prompt("refund from spotify")[0])

You are a financial assistant. Use the provided transactions to answer the user’s question.
Be accurate, concise, and only rely on the context. If the answer is not found, say so.

User question: refund from spotify

Transactions:
paypal refund of 42.15 USD from Spotify USA Inc. on 2025-07-22T00:00:00Z
paypal purchase of 42.15 USD to Spotify USA Inc. on 2025-07-15T00:00:00Z (Visa Ending in 1234)


In [ ]:
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a financial assistant. Use only the provided transactions to answer the user's question. "
                "If the answer is not found, say so. Be accurate, concise, and avoid making up information."
            )
        },
        {
            "role": "user",
            "content": generate_prompt("refund from spotify")[0]
        }
    ],
    temperature=0
)
print(response.choices[0].message.content)

The refund from Spotify is 42.15 USD, processed on 2025-07-22.
